In [ ]:
# ============================================
# SETUP KAGGLE ENVIRONMENT
# ============================================

# Install transformers
!pip install transformers -q

import os
import torch

# Check GPU
print('=' * 50)
print('ENVIRONMENT CHECK')
print('=' * 50)

if torch.cuda.is_available():
    print(f'GPU Available: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')
else:
    print('WARNING: GPU not available!')

# Find dataset
KAGGLE_INPUT = '/kaggle/input'
DATA_PATH = None

print(f'\nSearching for dataset...')
for root, dirs, files in os.walk(KAGGLE_INPUT):
    for f in files:
        if '5class_clean' in f and f.endswith('.csv'):
            DATA_PATH = os.path.join(root, f)
            print(f'Found: {DATA_PATH}')
            break

if DATA_PATH is None:
    print('\nDataset not found! Please upload gojek_reviews_5class_clean.csv')
    print('\nAvailable files in /kaggle/input:')
    for root, dirs, files in os.walk(KAGGLE_INPUT):
        for f in files:
            print(f'  - {os.path.join(root, f)}')

In [ ]:
# ============================================
# IMPORTS
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, 
    classification_report, confusion_matrix, f1_score
)
import random
import copy
import json

warnings.filterwarnings('ignore')

# Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

## 📊 1. Load & Explore Data

In [ ]:
# Load data
df = pd.read_csv(DATA_PATH)

# Label mapping untuk 5 kelas
LABEL_MAP = {
    'sangat_negatif': 0,
    'negatif': 1,
    'netral': 2,
    'positif': 3,
    'sangat_positif': 4
}
LABEL_NAMES = ['sangat_negatif', 'negatif', 'netral', 'positif', 'sangat_positif']
NUM_CLASSES = 5

# Create label column
df['label'] = df['sentiment_label'].map(LABEL_MAP)

print('=' * 60)
print('DATA OVERVIEW')
print('=' * 60)
print(f'Total samples: {len(df):,}')
print(f'\nColumns: {df.columns.tolist()}')
print(f'\nSentiment Distribution:')
print(df['sentiment_label'].value_counts())

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#e74c3c', '#f39c12', '#95a5a6', '#3498db', '#2ecc71']
sentiment_counts = df['sentiment_label'].value_counts().reindex(LABEL_NAMES)

axes[0].bar(range(5), sentiment_counts.values, color=colors)
axes[0].set_xticks(range(5))
axes[0].set_xticklabels(LABEL_NAMES, rotation=45, ha='right')
axes[0].set_title('Sentiment Distribution (5 Classes)')
axes[0].set_ylabel('Count')

axes[1].pie(sentiment_counts.values, labels=LABEL_NAMES, autopct='%1.1f%%', colors=colors)
axes[1].set_title('Sentiment Percentage')

plt.tight_layout()
plt.show()

# Sample reviews
print('\nSample Reviews:')
for label_name in LABEL_NAMES:
    sample = df[df['sentiment_label'] == label_name].sample(1).iloc[0]
    print(f'\n[{label_name.upper()}] Rating {sample["rating"]}:')
    print(f'   "{sample["review"][:80]}..."')

## ⚖️ 2. Data Split (Stratified)

In [ ]:
# Stratified split: Train (70%), Val (15%), Test (15%)
train_df, temp_df = train_test_split(
    df, test_size=0.3, random_state=42, stratify=df['label']
)

val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df['label']
)

print('=' * 60)
print('DATA SPLIT')
print('=' * 60)
print(f'Training set: {len(train_df):,} samples ({len(train_df)/len(df)*100:.1f}%)')
print(f'Validation set: {len(val_df):,} samples ({len(val_df)/len(df)*100:.1f}%)')
print(f'Test set: {len(test_df):,} samples ({len(test_df)/len(df)*100:.1f}%)')

print('\nDistribution per split:')
for name, data in [('Train', train_df), ('Val', val_df), ('Test', test_df)]:
    dist = data['label'].value_counts(normalize=True).sort_index() * 100
    print(f'{name}: ' + ' | '.join([f'{LABEL_NAMES[i][:4]}: {dist[i]:.1f}%' for i in range(5)]))

## 🔧 3. Load IndoBERT Tokenizer

In [ ]:
# Load IndoBERT tokenizer
MODEL_NAME = 'indobenchmark/indobert-base-p1'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# Analyze text lengths
text_lengths = df['review'].apply(lambda x: len(tokenizer.encode(str(x), add_special_tokens=True)))

print('=' * 60)
print('TEXT LENGTH ANALYSIS')
print('=' * 60)
print(f'Min tokens: {text_lengths.min()}')
print(f'Max tokens: {text_lengths.max()}')
print(f'Mean tokens: {text_lengths.mean():.1f}')
print(f'95th percentile: {text_lengths.quantile(0.95):.0f}')

MAX_LEN = min(int(text_lengths.quantile(0.95)) + 10, 128)
print(f'\nUsing MAX_LEN = {MAX_LEN}')

plt.figure(figsize=(10, 4))
plt.hist(text_lengths, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=MAX_LEN, color='r', linestyle='--', label=f'MAX_LEN = {MAX_LEN}')
plt.xlabel('Token Count')
plt.ylabel('Frequency')
plt.title('Distribution of Text Lengths')
plt.legend()
plt.show()

## 📦 4. Dataset Class

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, reviews, labels, tokenizer, max_len, augment=False):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.augment = augment
    
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, idx):
        review = str(self.reviews[idx])
        label = self.labels[idx]
        
        # No augmentation - let model learn the actual data first
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Create datasets
train_dataset = SentimentDataset(
    train_df['review'].values, train_df['label'].values,
    tokenizer, MAX_LEN, augment=True
)
val_dataset = SentimentDataset(
    val_df['review'].values, val_df['label'].values,
    tokenizer, MAX_LEN, augment=False
)
test_dataset = SentimentDataset(
    test_df['review'].values, test_df['label'].values,
    tokenizer, MAX_LEN, augment=False
)

# Data loaders
# Create datasets (no augmentation for cleaner learning)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    tokenizer, MAX_LEN, augment=False
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f'Train batches: {len(train_loader)}')
print(f'Val batches: {len(val_loader)}')
print(f'Test batches: {len(test_loader)}')

## 🏗️ 5. Model Architecture

In [ ]:
class IndoBERTSentiment5Class(nn.Module):
    def __init__(self, model_name, num_classes=5, dropout_rate=0.1, freeze_bert_layers=0):
        super(IndoBERTSentiment5Class, self).__init__()
        
        self.bert = BertModel.from_pretrained(model_name)
        self.hidden_size = self.bert.config.hidden_size
        
        # Freeze lower BERT layers (0 = unfreeze all for better learning)
        if freeze_bert_layers > 0:
            for param in self.bert.embeddings.parameters():
                param.requires_grad = False
            for i in range(freeze_bert_layers):
                for param in self.bert.encoder.layer[i].parameters():
                    param.requires_grad = False
        
        # Simpler classifier - less prone to underfitting
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.hidden_size, num_classes)
        
        self._init_weights()
    
    def _init_weights(self):
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Initialize model - UNFREEZE all layers for better learning
model = IndoBERTSentiment5Class(
    MODEL_NAME, num_classes=NUM_CLASSES, dropout_rate=0.1, freeze_bert_layers=0
)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print('=' * 60)
print('MODEL ARCHITECTURE')
print('=' * 60)
print(f'Total parameters: {total_params:,}')
print(f'Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.1f}%)')
print(f'Frozen parameters: {total_params - trainable_params:,}')
print('\nNote: All BERT layers are TRAINABLE for maximum learning capacity')

## 📉 6. Loss Function (Focal Loss + Label Smoothing)

In [ ]:
# Use simple CrossEntropyLoss with label smoothing
# Focal Loss can cause underfitting when model hasn't learned well yet

class LabelSmoothingLoss(nn.Module):
    def __init__(self, num_classes=5, smoothing=0.1):
        super().__init__()
        self.num_classes = num_classes
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing
    
    def forward(self, inputs, targets):
        log_probs = F.log_softmax(inputs, dim=1)
        
        # Create smoothed labels
        with torch.no_grad():
            true_dist = torch.zeros_like(log_probs)
            true_dist.fill_(self.smoothing / (self.num_classes - 1))
            true_dist.scatter_(1, targets.unsqueeze(1), self.confidence)
        
        return torch.mean(torch.sum(-true_dist * log_probs, dim=1))

# Simple CrossEntropy for initial training
criterion = nn.CrossEntropyLoss()

print('Loss: CrossEntropyLoss (simple, effective for learning)')
print('\nNote: Using simple loss to help model learn better first')

## ⚙️ 7. Training Configuration

In [ ]:
# Hyperparameters - OPTIMIZED FOR BETTER LEARNING
EPOCHS = 15  # More epochs to learn
LEARNING_RATE = 3e-5  # Slightly higher LR
WEIGHT_DECAY = 0.01
WARMUP_RATIO = 0.1
MAX_GRAD_NORM = 1.0
PATIENCE = 5  # More patience

# Optimizer with different LR for BERT and classifier
bert_params = list(model.bert.parameters())
classifier_params = list(model.classifier.parameters())

optimizer = AdamW([
    {'params': bert_params, 'lr': LEARNING_RATE},
    {'params': classifier_params, 'lr': LEARNING_RATE * 10}  # Higher LR for classifier
], weight_decay=WEIGHT_DECAY, eps=1e-8)

# Scheduler
total_steps = len(train_loader) * EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps
)

print('=' * 60)
print('TRAINING CONFIG')
print('=' * 60)
print(f'Epochs: {EPOCHS}')
print(f'Batch size: {BATCH_SIZE}')
print(f'Learning rate (BERT): {LEARNING_RATE}')
print(f'Learning rate (Classifier): {LEARNING_RATE * 10}')
print(f'Weight decay: {WEIGHT_DECAY}')
print(f'Warmup steps: {warmup_steps}')
print(f'Gradient clipping: {MAX_GRAD_NORM}')
print(f'Early stopping patience: {PATIENCE}')

## 🚀 8. Training Loop

In [ ]:
def train_epoch(model, data_loader, criterion, optimizer, scheduler, device, max_grad_norm):
    model.train()
    total_loss = 0
    predictions, actual_labels = [], []
    
    progress_bar = tqdm(data_loader, desc='Training', leave=False)
    
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        _, preds = torch.max(outputs, dim=1)
        predictions.extend(preds.cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())
        
        progress_bar.set_postfix({'loss': loss.item()})
    
    avg_loss = total_loss / len(data_loader)
    accuracy = accuracy_score(actual_labels, predictions)
    f1 = f1_score(actual_labels, predictions, average='macro')
    
    return avg_loss, accuracy, f1

def eval_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    predictions, actual_labels = [], []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc='Evaluating', leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(data_loader)
    accuracy = accuracy_score(actual_labels, predictions)
    f1 = f1_score(actual_labels, predictions, average='macro')
    
    return avg_loss, accuracy, f1, predictions, actual_labels

In [ ]:
# Training with Early Stopping
print('=' * 60)
print('STARTING TRAINING')
print('=' * 60)

best_val_f1 = 0
best_model_state = None
patience_counter = 0
history = {
    'train_loss': [], 'train_acc': [], 'train_f1': [],
    'val_loss': [], 'val_acc': [], 'val_f1': []
}

for epoch in range(EPOCHS):
    print(f'\nEpoch {epoch + 1}/{EPOCHS}')
    print('-' * 40)
    
    train_loss, train_acc, train_f1 = train_epoch(
        model, train_loader, criterion, optimizer, scheduler, device, MAX_GRAD_NORM
    )
    
    val_loss, val_acc, val_f1, _, _ = eval_model(model, val_loader, criterion, device)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['train_f1'].append(train_f1)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    
    print(f'Train - Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | F1: {train_f1:.4f}')
    print(f'Val   - Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}')
    
    # Overfitting check
    overfit_gap = train_acc - val_acc
    if overfit_gap > 0.1:
        print(f'WARNING: Overfitting detected! Gap: {overfit_gap:.4f}')
    
    # Early stopping
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_model_state = copy.deepcopy(model.state_dict())
        patience_counter = 0
        print(f'New best model! Val F1: {val_f1:.4f}')
    else:
        patience_counter += 1
        print(f'No improvement. Patience: {patience_counter}/{PATIENCE}')
        
        if patience_counter >= PATIENCE:
            print(f'\nEarly stopping at epoch {epoch + 1}')
            break

model.load_state_dict(best_model_state)
print(f'\nLoaded best model with Val F1: {best_val_f1:.4f}')

## 📊 9. Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
epochs_range = range(1, len(history['train_loss']) + 1)

# Loss
axes[0].plot(epochs_range, history['train_loss'], 'b-o', label='Train')
axes[0].plot(epochs_range, history['val_loss'], 'r-o', label='Val')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(epochs_range, history['train_acc'], 'b-o', label='Train')
axes[1].plot(epochs_range, history['val_acc'], 'r-o', label='Val')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# F1
axes[2].plot(epochs_range, history['train_f1'], 'b-o', label='Train')
axes[2].plot(epochs_range, history['val_f1'], 'r-o', label='Val')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('F1 Score')
axes[2].set_title('F1 Score')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history_5class.png', dpi=150)
plt.show()

# Overfitting analysis
gap = history['train_acc'][-1] - history['val_acc'][-1]
print('\nOVERFITTING ANALYSIS')
print('=' * 40)
print(f'Final Train Acc: {history["train_acc"][-1]:.4f}')
print(f'Final Val Acc: {history["val_acc"][-1]:.4f}')
print(f'Gap: {gap:.4f}')
print(f'Status: {"NOT Overfitting" if gap < 0.05 else "Check overfitting" if gap < 0.1 else "OVERFITTING!"}')

## 🧪 10. Test Set Evaluation

In [ ]:
# Evaluate on test set
print('=' * 60)
print('TEST SET EVALUATION')
print('=' * 60)

test_loss, test_acc, test_f1, test_preds, test_labels = eval_model(
    model, test_loader, criterion, device
)

print(f'\nTest Results:')
print(f'  Loss: {test_loss:.4f}')
print(f'  Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)')
print(f'  Macro F1: {test_f1:.4f}')

print('\nCLASSIFICATION REPORT')
print('=' * 60)
print(classification_report(test_labels, test_preds, target_names=LABEL_NAMES, digits=4))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(test_labels, test_preds)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=LABEL_NAMES, yticklabels=LABEL_NAMES, ax=axes[0])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title('Confusion Matrix (Counts)')

sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=LABEL_NAMES, yticklabels=LABEL_NAMES, ax=axes[1])
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
axes[1].set_title('Confusion Matrix (Normalized)')

plt.tight_layout()
plt.savefig('confusion_matrix_5class.png', dpi=150)
plt.show()

# Per-class accuracy
print('\nPER-CLASS ACCURACY')
print('=' * 40)
for i, label in enumerate(LABEL_NAMES):
    acc = cm[i, i] / cm[i].sum()
    print(f'{label}: {acc:.2%} ({cm[i, i]}/{cm[i].sum()})')

## 💾 11. Save Model

In [ ]:
# Save model to Kaggle output
OUTPUT_DIR = '/kaggle/working'

model_path = f'{OUTPUT_DIR}/indobert_sentiment_5class_best.pt'
torch.save({
    'model_state_dict': model.state_dict(),
    'model_config': {
        'model_name': MODEL_NAME,
        'num_classes': NUM_CLASSES,
        'max_len': MAX_LEN,
        'label_map': LABEL_MAP,
        'label_names': LABEL_NAMES
    },
    'training_history': history,
    'test_metrics': {'accuracy': test_acc, 'f1_score': test_f1, 'loss': test_loss}
}, model_path)

print(f'Model saved to: {model_path}')

# Save config
config = {
    'model_name': MODEL_NAME,
    'num_classes': NUM_CLASSES,
    'max_len': MAX_LEN,
    'batch_size': BATCH_SIZE,
    'epochs_trained': len(history['train_loss']),
    'test_accuracy': test_acc,
    'test_f1': test_f1,
    'label_map': LABEL_MAP,
    'label_names': LABEL_NAMES
}

with open(f'{OUTPUT_DIR}/training_config_5class.json', 'w') as f:
    json.dump(config, f, indent=2)

print(f'Config saved to: {OUTPUT_DIR}/training_config_5class.json')

## 🔮 12. Prediction Demo

In [ ]:
def predict_sentiment(text, model, tokenizer, device, max_len=MAX_LEN):
    model.eval()
    
    encoding = tokenizer.encode_plus(
        text, add_special_tokens=True, max_length=max_len,
        padding='max_length', truncation=True,
        return_attention_mask=True, return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = F.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probs, dim=1)
    
    return LABEL_NAMES[predicted.item()], confidence.item(), \
           {name: probs[0][i].item() for i, name in enumerate(LABEL_NAMES)}

# Test predictions
print('=' * 60)
print('SAMPLE PREDICTIONS')
print('=' * 60)

test_texts = [
    "Aplikasi ini sangat bagus dan membantu sekali!",
    "Lumayan lah, cukup membantu",
    "Biasa aja, tidak ada yang istimewa",
    "Kurang bagus, sering error",
    "Aplikasi sampah! Sangat mengecewakan!"
]

for text in test_texts:
    label, conf, probs = predict_sentiment(text, model, tokenizer, device)
    print(f'\n"{text[:50]}..."')
    print(f'  -> {label.upper()} ({conf:.2%})')

## 📈 13. Final Summary

In [ ]:
print('=' * 60)
print('FINAL SUMMARY')
print('=' * 60)

print(f'''
MODEL PERFORMANCE:
  - Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)
  - Test Macro F1: {test_f1:.4f}

TRAINING OPTIMIZATIONS APPLIED:
  - Full BERT fine-tuning (all layers trainable)
  - Simple CrossEntropyLoss (better initial learning)
  - Differential Learning Rates (BERT: 3e-5, Classifier: 3e-4)
  - Larger Batch Size (32)
  - More Epochs (15) with patience (5)
  - Lower Dropout (0.1)
  - Simpler classifier head
  - Weight Decay (0.01)
  - Learning Rate Warmup
  - Gradient Clipping (1.0)

OVERFITTING CHECK:
  - Train Acc: {history["train_acc"][-1]:.4f}
  - Val Acc: {history["val_acc"][-1]:.4f}
  - Gap: {history["train_acc"][-1] - history["val_acc"][-1]:.4f}
  - Status: {"Good" if abs(history["train_acc"][-1] - history["val_acc"][-1]) < 0.1 else "Check!"}

SAVED FILES:
  - indobert_sentiment_5class_best.pt
  - training_config_5class.json
  - training_history_5class.png
  - confusion_matrix_5class.png
''')

print('Training Complete!')